In [1]:
import $file.rec8stdlib
import rec8stdlib._

Compiling /home/achorn/Desktop/csci/fall2019/PPL/week10/recitation_week8/rec8stdlib.sc

import $file.$         

import rec8stdlib._

# Recitation N

In this recitation we will get some practice with the basic parser combinators.

## The Parser Type

We define parsers below with the following Grammar:

$$
\large \text{type }\textbf{Parser}\ s\ d = (\ (\textbf{List}\ s) \rightarrow\ \textbf{List}\ (d, (\textbf{List}\ s))\ )
$$

We could also write the $\textbf{List}$ type as square brackets. So we would write $\textbf{List}\ A $ as $[A]$ which may be easier to read:

$$
\large \text{type }\textbf{Parser}\ s\ d =  (\ [s] \rightarrow\ [(d, [s])]\ )
$$

As a refresher, let's break down each part of this type.

$$
\large{ \text{type }\textbf{Parser}\ \color{#1196cc}s\ \color{#cc8511}d =(\ \color{#27dc3a}{[s]\rightarrow\ [(d, [s])]}\ )}
$$

* $\color{#1196cc}s$ - The type of the symbols the parser is reading in. Usually this will be characters but could also be any other data type.
* $\color{#cc8511}d$ - The type of the data we are returning from the parse. This is the structure we are trying to build up. Later this will be Lettuce Expressions.
* $\color{#27dc3a}{[s]\rightarrow\ [(d, [s])]}$ - The parsing function. This is any function that takes a list of symbols(such as a string) and returns a list of success parses. A successful parse is any tuple of the parsed structure $d$ and the rest of the list that still needs to be parsed


In [2]:
// Parser type
type Parser[S,D] = List[S] => List[(D, List[S])]

defined type Parser

## The Primitives

We will define three primitive parsers that we will use to build up all of the other parsers we will need. These are:

* `char` Takes a character as an argument and parses that character
* `success` Takes any element of type $D$ and returns a parser of that type
* `failure` Unsuccesful parses

### Char
 
The `char` primitive is a parser for the provided character `c`. This can be any character that is included in Scala's definition. We will use this as the primary building block for all of our parsers going forward.

In [8]:
// char
def parse_a(input : List[Char]) = 
    input match{
        case Empty => Empty
        case Cons(c ,cs) => char_eq(c,'a') match{
            case True => singleton(('a',cs))
            case False => Empty
        }
    }
parse_a(string_to_list("abc"))

def char(my_char : Char) : Parser[Char, Char] = (input : List[Char]) => input match{
    case Empty => Empty
    case Cons(c ,cs) => char_eq(my_char,c) match{
            case True => singleton((my_char ,cs))
            case False => Empty
        }
}

char('c')(string_to_list("cabc"))

defined function parse_a
res7_1: List[(Char, List[Char])] = Cons(
  ('a', Cons('b', Cons('c', Empty))),
  Empty
)
defined function char
res7_3: List[(Char, List[Char])] = Cons(
  ('c', Cons('a', Cons('b', Cons('c', Empty)))),
  Empty
)

### Success

This is a fairly simples primitive that acts as a pass-through. It will just wrap up its argument into a successful parse. This will be useful when returning results inside of the `bind` combinator.

In [17]:
// success
def success[S,D](x : D) : Parser[S,D] =
    (input : List[S]) =>{
        singleton( (x, input) )
    }

defined function success

### Failure

This is the dual of `success` and is used any time we have an unsuccesful parse

In [9]:
// failure

def failure[S,D](): Parser[S,D] =
    (input : List[S]) => 
        Empty

defined function failure

## The Combinators

We will use these primitives with _combinators_(A name that only a mathmetician could make up) to create our parsers. There are two basic combinators from which we will build all of the others:

### Choice

`Choice` represents a case where you have two parsers and want to combine them in an either/or way. If you have a parser that recognizes numbers and another that recognizes words you could combine them to recognize both numbers and words.

In [10]:
// choose
def choose[S,D](p1 : Parser[S,D], p2 : Parser[S,D]): Parser[S,D] =
    (input : List[S]) =>{
        val parse1 = p1(input)
        val parse2 = p2(input)
        append(p1(input), p2(input))
    }

defined function choose

# Examples

Note: To convert a string to a list of characters call the `string_to_list` function

### 1
Write a parser that accepts either a string beginning with `'a'` or `'z'`

In [12]:
val p_a = char('a')
val p_z = char('z')
val p1 = choose(p_a, p_z)

val ex1 = string_to_list("abc")
val ex2 = string_to_list("zyx")
val ex_bad = string_to_list("dog")

p1(ex1)

p_a: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd7$Helper$$Lambda$3220/0x0000000840d4f840@7205bf50
p_z: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd7$Helper$$Lambda$3220/0x0000000840d4f840@777a323d
p1: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd9$Helper$$Lambda$3231/0x0000000840d56840@6d49939f
ex1: List[Char] = Cons('a', Cons('b', Cons('c', Empty)))
ex2: List[Char] = Cons('z', Cons('y', Cons('x', Empty)))
ex_bad: List[Char] = Cons('d', Cons('o', Cons('g', Empty)))
res11_6: List[(Char, List[Char])] = Cons(
  ('a', Cons('b', Cons('c', Empty))),
  Empty
)

### 2
Write a parser that accepts any digit (0-9)

In [17]:
// val p1 = choose(char('0'),  choose(char('1'), 
//                             choose(char('2'), 
//                             choose(char('3'), 
//                             choose(char('4'),
//                             choose(char('5'),
//                             choose(char('6'), 
//                             choose(char('7'), 
//                             choose(char('8'),char('9'))))))))))

val num_parsers = 
Cons(char('1'),
Cons(char('2'),
Cons(char('3'),
Cons(char('4'),
Cons(char('5'),
Cons(char('6'),
Cons(char('7'),
Cons( char('8'),char('9') ))))))))

val pdigits = ???

cmd17.sc:9: type mismatch;
 found   : ammonite.$sess.cmd7.wrapper.cmd1.Parser[Char,Char]
    (which expands to)  ammonite.$sess.cmd1.wrapper.rec8stdlib.List[Char] => ammonite.$sess.cmd1.wrapper.rec8stdlib.List[(Char, ammonite.$sess.cmd1.wrapper.rec8stdlib.List[Char])]
 required: cmd17.this.rec8stdlib.List[?]
Cons( char('8'),char('9') ))))))))
                    ^Compilation Failed

: 

In [17]:
assert(pdigits(string_to_list("4")) == Cons(('4', Empty),Empty))
assert(pdigits(string_to_list("9sd")) == Cons(('9', Cons('s', Cons('d', Empty))),Empty))
assert(pdigits(string_to_list("214")) == Cons(('2', Cons('1', Cons('4', Empty))),Empty))
assert(pdigits(string_to_list("d")) == Empty)
assert(pdigits(string_to_list("d3443")) == Empty)

cmd17.sc:1: not found: value pdigits
val res17_0 = assert(pdigits(string_to_list("4")) == Cons(('4', Empty),Empty))
                     ^cmd17.sc:2: not found: value pdigits
val res17_1 = assert(pdigits(string_to_list("9sd")) == Cons(('9', Cons('s', Cons('d', Empty))),Empty))
                     ^cmd17.sc:3: not found: value pdigits
val res17_2 = assert(pdigits(string_to_list("214")) == Cons(('2', Cons('1', Cons('4', Empty))),Empty))
                     ^cmd17.sc:4: not found: value pdigits
val res17_3 = assert(pdigits(string_to_list("d")) == Empty)
                     ^cmd17.sc:5: not found: value pdigits
val res17_4 = assert(pdigits(string_to_list("d3443")) == Empty)
                     ^Compilation Failed

: 

### 3
Write 4 parsers which do the following when given the string `"abcd"`
1. One that fails to parse
2. One that produces a single successful parse
3. One that produces 3 successful parses (it's ok if they're the same as long as there are 3 results in the list)
4. One that produces 32 results (think about how to do this efficiently)

In [20]:
val x = string_to_list("abcd")

val parser1 = failure[Char,Char]
val parser2 = success[Char,Char]('!')
val parser3 = choose(char('a'), choose(char('a'), char('a')))
val p1 = choose(char('a'),char('a'))
val p2 = choose(p1,p1)
val p3 = choose(p2,p2)
val p4 = choose(p3,p3)
val parser4 = choose(p4,p4)

x: List[Char] = Cons('a', Cons('b', Cons('c', Cons('d', Empty))))
parser1: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd8$Helper$$Lambda$3349/0x0000000840d9f840@32131630
parser2: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd16$Helper$$Lambda$3416/0x0000000840dd4840@671190ca
parser3: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd9$Helper$$Lambda$3231/0x0000000840d56840@5a3da04e
p1: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd9$Helper$$Lambda$3231/0x0000000840d56840@2380745
p2: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd9$Helper$$Lambda$3231/0x0000000840d56840@60623fdb
p3: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd9$Helper$$Lambda$3231/0x0000000840d56840@669f2caa
p4: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd9$Helper$$Lambda$3231/0x0000000840d56840@25a3432b
parser4: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd9$Helper$$Lambda$3231/0x0000000840d56840@2d257ca5

In [21]:
assert(length(parser1(x)) == Zero)
assert(length(parser2(x)) == one)
assert(length(parser3(x)) == three)
assert(length(parser4(x)) == nat_pow(five, two))